In [ ]:
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree

from skfp.filters import BeyondRo5Filter, BMSFilter, BrenkFilter, FAF4DruglikeFilter, FAF4LeadlikeFilter, GhoseFilter, GlaxoFilter, GSKFilter
from skfp.filters import HaoFilter, InpharmaticaFilter, LINTFilter, LipinskiFilter, MLSMRFilter, MolecularWeightFilter, NIBRFilter, NIHFilter
from skfp.filters import OpreaFilter, PAINSFilter, PfizerFilter, REOSFilter, RuleOfFourFilter, RuleOfThreeFilter, RuleOfTwoFilter, RuleOfVeberFilter
from skfp.filters import RuleOfXuFilter, SureChEMBLFilter, TiceHerbicidesFilter, TiceInsecticidesFilter, ValenceDiscoveryFilter, ZINCBasicFilter, ZINCDruglikeFilter


In [3]:

#models for regression
model = [
    svm.SVR(),
    MLPRegressor(random_state=1, max_iter=500),
    LogisticRegression(random_state=0),
    linear_model.BayesianRidge(),
    make_pipeline(StandardScaler(), SGDRegressor(max_iter=1000, tol=1e-3)),
    KNeighborsRegressor(n_neighbors=2),
    tree.DecisionTreeRegressor()

]